# 📕 02 - Transform Raw Data into Time Series Data

## Introduction

In the preceding notebook, `01_load_and_validate_raw_data.ipynb`, we embarked on the initial journey of ingesting and pre-processing the TLC's raw taxi ride data. With the refined dataset in hand, our next mission is to shape this information into a structured time-series format.

The prime objective? To aggregate the number of rides according to two pivotal axes: `pickup_hour` and `pickup_location_id`, ensure continuity by addressing potential gaps in houw dataset.

## Notebook Overview:

1. **Validated Dataset Retrieval**: Begin by importing the validated dataset.
2. **Hourly Binning**: Isolate the exact timestamps of each ride and group them into hourly segments. This approach simplifies our dataset by representing each ride by the hour it commenced, rather than its precise starting moment. 
3. **Aggregation**: Compile the data by both hour and location, yielding a concise count of rides for every unique `pickup_hour`-`pickup_location_id` combination.
4. **Ensuring Continuity**: Time-series data thrives on regularity. Address potential gaps in our dataset by ensuring every location possesses entries for the entire temporal span.
5. **Visualization**: To validate and explore our time-series data further, draw a dynamic line chart depicting the frequency of rides over time for select pickup locations.

In [1]:
# import libraries
import pandas as pd
from tqdm import tqdm
from typing import Optional, List
import plotly.express as px

# Load the validated raw data and check the first 20 entries
rides = pd.read_parquet('../data/transformed/validated_rides_2022_01.parquet')
rides.head(20)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
5,2022-01-01 00:40:15,138
6,2022-01-01 00:20:50,233
7,2022-01-01 00:13:04,238
8,2022-01-01 00:30:02,166
9,2022-01-01 00:48:52,236


In [2]:
# Here, we transform the precise `pickup_datetime` into hourly intervals
rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
rides

,pickup_datetime,pickup_location_id,pickup_hour
0,2022-01-01 00:35:40,142,2022-01-01 00:00:00
1,2022-01-01 00:33:43,236,2022-01-01 00:00:00
2,2022-01-01 00:53:21,166,2022-01-01 00:00:00
3,2022-01-01 00:25:21,114,2022-01-01 00:00:00
4,2022-01-01 00:36:48,68,2022-01-01 00:00:00
...,...,...,...
2463926,2022-01-31 23:36:53,90,2022-01-31 23:00:00
2463927,2022-01-31 23:44:22,107,2022-01-31 23:00:00
2463928,2022-01-31 23:39:00,113,2022-01-31 23:00:00
2463929,2022-01-31 23:36:42,148,2022-01-31 23:00:00


In [3]:
# aggregate the data
agg_rides = rides.groupby(['pickup_hour', 'pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)

agg_rides

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,4,11
1,2022-01-01 00:00:00,7,6
2,2022-01-01 00:00:00,10,1
3,2022-01-01 00:00:00,12,2
4,2022-01-01 00:00:00,13,12
...,...,...,...
66863,2022-01-31 23:00:00,261,4
66864,2022-01-31 23:00:00,262,8
66865,2022-01-31 23:00:00,263,26
66866,2022-01-31 23:00:00,264,24


In [4]:

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    """
    Add missing hourly slots to a DataFrame of ride counts, filling them with zeros.

    Parameters:
    - agg_rides (pd.DataFrame): A DataFrame containing ride counts, with columns 'pickup_hour', 'pickup_location_id', and 'rides'.

    Returns:
    - pd.DataFrame: A new DataFrame with the same columns as `agg_rides`, but with additional rows for any missing hourly slots, filled with zeros.

    Example:
    >>> rides = pd.DataFrame({'pickup_hour': ['2022-01-01 00:00:00', '2022-01-01 02:00:00'], 'pickup_location_id': [1, 1], 'rides': [10, 20]})
    >>> add_missing_slots(rides)
    pickup_hour         pickup_location_id   rides
    2022-01-01 00:00:00 1                    10
    2022-01-01 01:00:00 1                    0
    2022-01-01 02:00:00 1                    20
    """
    
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()

    for location_id in tqdm(location_ids):
        agg_rides_i = agg_rides.loc[agg_rides['pickup_location_id'] == location_id, ['pickup_hour', 'rides']]
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        agg_rides_i['pickup_location_id'] = location_id

        output = pd.concat([output, agg_rides_i])

    output = output.reset_index().rename(columns={'index': 'pickup_hour'})

    return output


In [5]:
agg_rides_all_slots = add_missing_slots(agg_rides)

  0%|          | 0/257 [00:00<?, ?it/s]

100%|██████████| 257/257 [00:00<00:00, 379.52it/s]


In [6]:
def plot_rides(
        rides: pd.DataFrame,
        locations: Optional[List[int]] = None
) -> None:
    """
    Plot a line chart of the number of rides over time for specified pickup locations using Plotly.

    Parameters:
    - rides (pd.DataFrame): DataFrame containing ride information. It must have columns 'pickup_hour', 'rides', and 'pickup_location_id'.
    - locations (Optional[List[int]], default=None): A list of pickup_location_id values to be plotted. 
        If None, all locations in the 'rides' DataFrame will be plotted.

    Returns:
    None: Shows the plot using Plotly.
    
    Example:
    >>> plot_rides(df_rides, locations=[123, 456])
    """
    
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides

    fig = px.line(
        rides_to_plot,
        x='pickup_hour',
        y='rides',
        color='pickup_location_id',
        template='none'
    )

    fig.show()

# plot the rides for locations 4 and 43
plot_rides(agg_rides_all_slots, locations=[4, 43])


In [7]:
# save aggregated data to parquet
agg_rides_all_slots.to_parquet("../data/transformed/ts_data_rides_2022_01.parquet")